# Graphes avec Dijkstra et A*

Présenté par Hugo THOLLON, Nicolas HO, Naria SAVARY


## Introduction

A mesure que la technologie a évolué, et avec elle la mobilité de l'humanité, puis des individus. Cette mobilité accrue a amené de nouveaux problèmes. Là où, à une époque, monsieur et madame tout le monde n'allaient pas plus loin que la boulangerie du village, avec l'accès à la mobilité, notamment depuis la généralisation de l'automobile, les réseaux routiers se sont complexifiés, créant un besoin de trouver des chemins de plus en plus complexe, de plus en plus rapidement. Ainsi, les techniques qui servaient à choisir les routes ont dû évoluer, et avec l'ère de l'informatique, ces techniques sont devenus des algorithmes qui ont dû être optimisés pour satisfaire les exigences croissantes du réseau routier et de ses usagers.

Dans ce rapport nous allons présenter certains de ces algorithmes en commençant par l'algorithme de Dijkstra puis une de ses dérivée appelée A*. Nous finirons ce rapport en présentant HPA* (Hierarchical Pathfinding A*) une amélioration possible de A*.  

## Dijkstra

### Présentation

_D'après Wikipedia_, l'algorithme de Dijkstra permet de résoudre le problème du plus court chemin. Ce problème a plusieurs variantes. La plus simple est la suivante : étant donné un graphe non-orienté, dont les arêtes sont munies de poids, et deux sommets de ce graphe, trouver un chemin entre les deux sommets dans le graphe, de poids minimum. L'algorithme de Dijkstra permet de résoudre un problème plus général : le graphe peut être orienté, et l'on peut désigner un unique sommet, et demander d'avoir la liste des plus courts chemins pour tous les autres nœuds du graphe.

Dans un graphe, le poids d’une arête représente la “coût” associé au passage entre deux sommets. Ce coût peut modéliser de nombreuses choses selon le contexte du problème : 
- une distance géographique, 
- un temps de trajet, 
- une consommation d’énergie, 
- un prix, 
- une difficulté.

L’algorithme de Dijkstra cherche alors à minimiser la somme de ces coûts. Choisir ce que représente le poids revient à définir ce que l’on veut optimiser dans le chemin trouvé.

Nous allons utiliser le terme "poids du nœud X" pour parler de la somme des poids des arêtes entre le nœud de départ D et X un nœud du graphe. Il peut exister plusieurs chemins entre D et X. Le poids d'un nœud fait réference au chemin avec le plus petit poids découvert pour l'instant.

Dans l'algorithme de Dijkstra, les poids sont forcément positifs. Pour des poids négatifs il est préferable d'utiliser l'[algorithme de Bellman-Ford](https://fr.wikipedia.org/wiki/Algorithme_de_Bellman-Ford) dont nous ne parlerons pas ici.  

Dans nos implémentations et pour cette présentation, nous considèrerons que les **graphes sont non-orientés** et que les **poids** représentent la **distance en mètre** entre 2 nœuds.

### Explication avec un exemple

#### Théorie

Dans cette présentation, nous allons utiliser le graphe suivant pour expliquer le fonctionnement de Dijkstra et A* : 

<center>

<img src="./assets/graph_example_without_weight.png" alt="image du graphe d'exemple" style="background-color:white; max-width:300px;" />

_**figure 1:** graphe d'exemple_

</center>

Dans ce graphe, nous avons représenté le poids de chaque arête. Au fur et à mesure que nous compléterons le graphe nous renseignerons le poids de chaque nœud à la place de **00**.

Il est possible d'éxecuter l'algorithme de Dijkstra avec un tableau :  
Chaque étape correspond à une ligne.  
Une **ligne** donne les distances des sommets depuis le sommet de départ (càd le **poids des nœuds**).  
Une **colonne** donne l'évolution des poids d'un nœud.  

**A chaque étape** (Sur chaque ligne), le nœud avec le plus petit poids est **choisi** et souligné.  
**A l'étape suivante** le poids des nœuds adjacent (___NA___) au nœud choisi (___NC___) sont recalculés comme suit :

<center>

___Poids NA___ = ___Poids du NC___ + ___Poids Arête entre NA et NC___

</center>

Dans le tableau, si le nouveau ___Poids NA___ est inférieur au **poids** déjà présent, l'ancien poids est remplacé par le nouveau.

Une fois un nœud choisi, il ne peut plus être choisi. Pour rendre le tableau plus lisible, son poids dans le tableau est remplacé par **-**.

Si le poids d'un nœud n'est pas encore connu, son poids est noté ∞ (infini).

L'algorithme s'arrête quand le nœud choisi est le d'arrivée ou si tous les chemins ont été explorés.

#### Pratique

Pour montrer le fonctionnement de l'algorithme vous trouverez en dessous un GIF déroulant l'algorithme étape par étape avec le graphe présenté dans la figure 1 et un tableau qui s'actualise comme décrit dans la partie théorie.

Le code couleur du GIF est le suivant : 
- <span style="color:#CCFF99">Vert clair</span> : Départ.
- <span style="color:#66B2FF">Bleu</span> : Nœud découvert (càd que l'algorithme peut choisir) qui n'ont pas encore été choisis.
- <span style="color:#FF9999">Rouge rosé</span>/<span style="color:#E5CCFF">Rose-violet</span> : Arrivée avant et après avoir été découverte.
- <span style="color:#FFFF00">Jaune</span> : Le nœud choisi à une étape N.
- <span style="color:#00FF00">Vert</span> : Chemin final le plus court entre A et J.

<center>

<img src="./assets/dijkstra_steps.gif" alt="GIF expliquant l'algorithme de Dijkstra en plusieurs étapes" style="background-color:white; max-width:500px;" />

_**figure 2:** GIF expliquant l'algorithme de Dijkstra en plusieurs étapes_

</center>

Pour visualiser chaque étapes individuellement (sans le GIF) vous pouvez aller dans `./assets/dijkstra_steps/` et visualiser chaque image. 


### Le problème de Dijkstra

TODO